In [14]:
import os

# os.chdir("/scratch/project/tcr_neoantigen/misc/jaz/scripts")

from pathlib import Path
from pyfaidx import Fasta
from tqdm import tqdm

from _utils import (
    find,
    read_and_filter,
    create_result_list,
    extract_result,
    extract_exon_info,
    filter_exon_pos,
    get_sequences_indel,
    get_sequences_substitution,
    complementary_sequence,
    reverse_complement,
    flanking_lower_positions,
    # print_windows,
)

# define head paths
HG38FOLDER = Path(".../Reference/GRCh38.d1.vd1/fasta")
INPUTFOLDER = Path("/Users/Ali/Desktop/Long Covid/minigene_design/input")

In [16]:
# import reference and data
fasta_file = HG38FOLDER / "GRCh38.d1.vd1.dict"
gtf_file = HG38FOLDER / "gencode.v33.primary_assembly.annotation.gtf"
refgen = Fasta(filename=fasta_file)
exon_info = extract_exon_info(gtf_file)

FastaNotFoundError: Cannot read FASTA from file .../Reference/GRCh38.d1.vd1/fasta/GRCh38.d1.vd1.dict

In [ ]:
samples = [
    "2020135",
    "2020239_WO1",
    "2020246_NO1",
    "2020260_WO1",
    "2020281_WO1",
    "2021111_MO1",
    "DES001",
    "DES002",
    "DES002_001",
    "DES002_002",
    "DES010",
]

In [ ]:
results = {}
for sample in samples:
    file_input_path = (
        INPUTFOLDER
        / sample
        / "analyses"
        / sample
        / "05_vep"
        / "tables"
        / "high_confidence"
        / f"{sample}_hc_vep.txt"
    )
    results[sample] = read_and_filter(file_input_path)

for sample in samples:
    print(sample, results[sample].shape)

for sample in samples:
    print(sample, results[sample].VARIANT_CLASS.unique())

Exception ignored in: <function Faidx.__del__ at 0x111d6bba0>
Traceback (most recent call last):
  File "/Users/Ali/anaconda3/envs/minigenes/lib/python3.12/site-packages/pyfaidx/__init__.py", line 883, in __del__
    self.__exit__()
  File "/Users/Ali/anaconda3/envs/minigenes/lib/python3.12/site-packages/pyfaidx/__init__.py", line 889, in __exit__
    self.file.close()
    ^^^^^^^^^
AttributeError: 'Faidx' object has no attribute 'file'
Exception ignored in: <function Faidx.__del__ at 0x111d6bba0>
Traceback (most recent call last):
  File "/Users/Ali/anaconda3/envs/minigenes/lib/python3.12/site-packages/pyfaidx/__init__.py", line 883, in __del__
    self.__exit__()
  File "/Users/Ali/anaconda3/envs/minigenes/lib/python3.12/site-packages/pyfaidx/__init__.py", line 889, in __exit__
    self.file.close()
    ^^^^^^^^^
AttributeError: 'Faidx' object has no attribute 'file'
Exception ignored in: <function Faidx.__del__ at 0x111d6bba0>
Traceback (most recent call last):
  File "/Users/Ali/an

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Ali/Desktop/Long Covid/minigene_design/input/2020135/analyses/2020135/05_vep/tables/high_confidence/2020135_hc_vep.txt'

In [ ]:
for sample in samples:
    df = results[sample].copy()
    mut_dict = create_result_list(df)
    mutations = find(mut_dict["variant_class"], "insertion")
    if len(mutations) > 0:
        for mut in tqdm(mutations):
            ref_seqs, var_seqs = get_sequences_indel(
                mut_info=extract_result(mut_dict, mut),
                exon_info=exon_info,
                fasta=refgen,
            )
    mutations = find(mut_dict["variant_class"], "deletion")
    if len(mutations) > 0:
        for mut in tqdm(mutations):
            ref_seqs, var_seqs = get_sequences_indel(
                mut_info=extract_result(mut_dict, mut),
                exon_info=exon_info,
                fasta=refgen,
            )
    mutations = find(mut_dict["variant_class"], "SNV")
    if len(mutations) > 0:
        for mut in tqdm(mutations):
            ref_seqs, var_seqs = get_sequences_substitution(
                mut_info=extract_result(mut_dict, mut),
                exon_info=exon_info,
                fasta=refgen,
            )
    mutations = find(mut_dict["variant_class"], "substitution")
    if len(mutations) > 0:
        for mut in tqdm(mutations):
            ref_seqs, var_seqs = get_sequences_substitution(
                mut_info=extract_result(mut_dict, mut),
                exon_info=exon_info,
                fasta=refgen,
            )

100%|██████████| 34/34 [00:00<00:00, 5537.68it/s]


In [ ]:
seq = "TTCGCTCAAGGCTGTGGGTCCACCACCCCAAACCCCTCCACGAAGACACAGGGGCCTGCAGGCTGCCCGGCCAGCGGAGCCCACCCTACCCAGT"


In [ ]:
# situation 1: GAA GAC, CTT CTG, GTC TTC, GTC TTC, CAG AAG
dict1={"GAA" : "GAG",
"CTT" : "CTA",
"GTC" : "GTA",
"CAG" : "CAA"}
# situation 2: _GA AGAC, _CT TCT G, _GT CTT C, _CA GAA G
dict2={
"AGA" : "AGG",
"GGA" : "GGG",
"CGA" : "CGG",
"TGA" : "TAG",
"ACT" : "ACC",
"GCT" : "GCC",
"CCT" : "CCC",
"TCT" : "TCC",
"AGT" : "AGC",
"GGT" : "GGC",
"CGT" : "CGC",
"TGT" : "TGC",
"ACA" : "ACC",
"GCA" : "GCC",
"CCA" : "CCC",
"TCA" : "TCC"}

# situation 3: __G AAG AC, __C TT CTG, __G TCT TC, __C AGA AG
dict3={"AAG" : "AAA",
"ACG" : "ACT",
"AGG" : "AGA",
"TAG" : "TGA",
"TTG" : "TTA",
"TCG" : "TCT",
"CAG" : "CAA",
"CTG" : "CTA",
"CGG" : "CGA",
"GAG" : "GAA",
"GTG" : "GTA",
"GCG" : "GCT",
"GGG" : "GGA",
"AAC" : "AAT",
"ACC" : "ACT",
"AGC" : "AGT",
"ATC" : "ATT",
"TAC" : "TAT",
"TTC" : "TTT",
"TCC" : "TCT",
"TGC" : "TGT",
"CAC" : "CAT",
"CTC" : "CTT",
"CGC" : "CGT",
"CCC" : "CCT",
"GAC" : "GAT",
"GTC" : "GTT",
"GCC" : "GCT",
"GGC" : "GGT"}

# situation 4
# ATG -> Situation 2
# TGG -> Situation 2

In [ ]:


import re
for seq in df["minigenes"]:
	for x in ["GAAGAC","CTTCTG","GTCTTC","CAGAAG",]:
		if re.search(x, seq):
			# first need to find the location number of the sequence and see if that number is divisible by 3
			position = seq.find(x)
			if position % 3 == 0:
				# situation 1
				pos = position
				new_seq = seq[:pos] + dict1[seq[pos:pos+3]] + seq[pos+3:]
				print("x")
			else:
				if (position - 1) % 3 == 0:
					# situation 2
					pos = position - 1
					new_seq = seq[:pos] + dict2[seq[pos:pos+3]] + seq[pos+3:]
				else:
					# situation 3
					pos = position - 2
					if seq[pos:pos+3] in dict2:
						new_seq = seq[:pos] + dict3[seq[pos:pos+3]] + seq[pos+3:]
					else:
						# becomes situation 2
						pos = position - 1
						new_seq = seq[:pos] + dict2[seq[pos:pos+3]] + seq[pos+3:]

NameError: name 'df' is not defined